# Calendar Sync Configuration

This notebook provides an easy way to set up and configure your calendar sync. It will help you:

1. Connect external calendars (Outlook, iCloud, etc.) to your Google Calendar
2. Set up sync frequencies and time windows
3. Generate a configuration that can run continuously

## Setup

First, make sure you've installed the required packages and set up your Google Calendar API credentials. If you haven't done this yet, please refer to the README.md file for detailed instructions.

## Calendar Configuration

Use the form below to configure your calendar settings. You can add as many calendars as you need.

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import os

# Default settings
default_days_back = 30
default_days_forward = 60
default_sync_interval = 5

# Try to load existing configuration
config_file = 'calendar_config.json'
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        try:
            saved_calendars = json.load(f)
        except:
            saved_calendars = []
else:
    saved_calendars = []

# Pre-fill with examples if no configuration exists
if not saved_calendars:
    saved_calendars = [
        { 
            'url': 'https://outlook.office365.com/owa/calendar/70428bd059d14219bddf1221b4b3d621@ipsos.com/6764ef7293d54ee68e91b63fe5c4876910567899067948008408/calendar.ics',
            'calendarName': 'IPSOS',
            'daysBack': 30,
            'daysForward': 60,
            'syncInterval': 5
        },
        { 
            'url': 'https://ical.titan.email/feed/2973177/LfmH6NurIpULTZxvLvAW6dW7543wAgy3',
            'calendarName': 'Titan',
            'daysBack': 30,
            'daysForward': 60,
            'syncInterval': 5
        }
    ]

# Main calendar container
calendar_widgets = []

def create_calendar_form(url='', name='', days_back=default_days_back, days_forward=default_days_forward, sync_interval=default_sync_interval):
    """Create widgets for a single calendar configuration"""
    url_widget = widgets.Text(value=url, description='iCal URL:', style={'description_width': '120px'}, layout=widgets.Layout(width='90%'))
    name_widget = widgets.Text(value=name, description='Calendar Name:', style={'description_width': '120px'}, layout=widgets.Layout(width='50%'))
    days_back_widget = widgets.IntSlider(value=days_back, min=7, max=365, step=7, description='Days Back:', style={'description_width': '120px'})
    days_forward_widget = widgets.IntSlider(value=days_forward, min=7, max=365, step=7, description='Days Forward:', style={'description_width': '120px'})
    sync_interval_widget = widgets.IntSlider(value=sync_interval, min=1, max=60, step=1, description='Sync Interval (min):', style={'description_width': '120px'})
    
    remove_btn = widgets.Button(description='Remove', button_style='danger', layout=widgets.Layout(width='100px'))
    
    container = widgets.VBox([
        widgets.HBox([name_widget, remove_btn]),
        url_widget,
        widgets.HBox([days_back_widget, days_forward_widget]),
        sync_interval_widget,
        widgets.HTML(value='<hr style="height:2px;border-width:0;background-color:#ccc">')
    ])
    
    # Store all widgets together
    calendar_data = {
        'container': container,
        'url': url_widget,
        'name': name_widget,
        'days_back': days_back_widget,
        'days_forward': days_forward_widget,
        'sync_interval': sync_interval_widget,
        'remove_btn': remove_btn
    }
    
    calendar_widgets.append(calendar_data)
    
    # Handle removal
    def on_remove_clicked(b):
        calendar_widgets.remove(calendar_data)
        update_display()
    
    remove_btn.on_click(on_remove_clicked)
    
    return container

# Create main container and actions
main_container = widgets.VBox([])
add_btn = widgets.Button(description='Add Calendar', button_style='success')
save_btn = widgets.Button(description='Save Configuration', button_style='primary')
status_text = widgets.HTML(value='')

def update_display():
    """Update the display with all calendar widgets"""
    calendars = [widget['container'] for widget in calendar_widgets]
    main_container.children = calendars + [widgets.HBox([add_btn, save_btn]), status_text]

def on_add_clicked(b):
    """Handle adding a new calendar"""
    create_calendar_form()
    update_display()

def on_save_clicked(b):
    """Save the configuration to a file"""
    calendars_config = []
    
    for widget in calendar_widgets:
        calendars_config.append({
            'url': widget['url'].value,
            'calendarName': widget['name'].value,
            'daysBack': widget['days_back'].value,
            'daysForward': widget['days_forward'].value, 
            'syncInterval': widget['sync_interval'].value
        })
    
    # Save to file
    with open(config_file, 'w') as f:
        json.dump(calendars_config, f, indent=2)
    
    # Update run_calendar_sync.py
    with open('run_calendar_sync.py', 'r') as f:
        content = f.read()
    
    # Replace the calendars section
    calendars_json = json.dumps(calendars_config, indent=4).replace('\n', '\n    ')
    new_content = content.split('# Define your calendars here')[0] + '# Define your calendars here\ncalendars = ' + calendars_json + '\n' + content.split('def sync_calendar')[1]
    
    # Write back to file
    with open('run_calendar_sync.py', 'w') as f:
        f.write('#!/usr/bin/env python3\nimport importlib.util\nimport threading\nimport time\nimport os\nimport logging\nimport sys\n\n# Set up logging\nlogging.basicConfig(\n    level=logging.INFO, \n    format=\'%(asctime)s - %(name)s - %(levelname)s - %(message)s\',\n    handlers=[logging.FileHandler("calendar_sync_runner.log"), logging.StreamHandler()]\n)\nlogger = logging.getLogger(__name__)\n\n# Import the CalendarSync class from the script\nspec = importlib.util.spec_from_file_location("calendar_sync", "calendar_sync.py")\ncalendar_sync = importlib.util.module_from_spec(spec)\nspec.loader.exec_module(calendar_sync)\n\n# Define your calendars here\ncalendars = ' + calendars_json + '\n\ndef sync_calendar(calendar_config):\n    """Function to sync a single calendar in a separate thread"""\n    logger.info(f"Starting sync for calendar: {calendar_config[\'calendarName\']}")')
        f.write(content.split('def sync_calendar(calendar_config):')[1])
    
    status_text.value = f"<div style=\"color: green; font-weight: bold;\">Configuration saved ({len(calendars_config)} calendars configured)</div>"

# Set up event handlers
add_btn.on_click(on_add_clicked)
save_btn.on_click(on_save_clicked)

# Add existing calendars
for calendar in saved_calendars:
    create_calendar_form(
        url=calendar.get('url', ''),
        name=calendar.get('calendarName', ''),
        days_back=calendar.get('daysBack', default_days_back),
        days_forward=calendar.get('daysForward', default_days_forward),
        sync_interval=calendar.get('syncInterval', default_sync_interval)
    )

# Display the form
update_display()
display(main_container)

## Run the Sync Process

Once you've configured your calendars, you can use the button below to start the synchronization process. The sync will run in the background and continue until you stop it.

In [2]:
import sys
import subprocess
import threading
import time
from IPython.display import clear_output

sync_process = None
stop_log_thread = False
log_thread = None

def start_sync():
    global sync_process, stop_log_thread, log_thread
    
    if sync_process is not None:
        status_output.value = "<div style='color: orange; font-weight: bold;'>Sync is already running</div>"
        return
    
    # Start the sync process
    sync_process = subprocess.Popen([sys.executable, 'run_calendar_sync.py'], 
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.STDOUT,
                                   universal_newlines=True,
                                   bufsize=1)
    
    # Update UI
    start_btn.disabled = True
    stop_btn.disabled = False
    status_output.value = "<div style='color: green; font-weight: bold;'>Sync started</div>"
    log_output.value = "Starting calendar sync...\n"
    
    # Start log monitoring thread
    stop_log_thread = False
    log_thread = threading.Thread(target=monitor_logs)
    log_thread.daemon = True
    log_thread.start()

def stop_sync():
    global sync_process, stop_log_thread
    
    if sync_process is None:
        status_output.value = "<div style='color: orange; font-weight: bold;'>No sync process is running</div>"
        return
    
    # Stop the process
    sync_process.terminate()
    sync_process = None
    stop_log_thread = True
    
    # Update UI
    start_btn.disabled = False
    stop_btn.disabled = True
    status_output.value = "<div style='color: red; font-weight: bold;'>Sync stopped</div>"
    log_output.value += "\nSync process stopped.\n"

def monitor_logs():
    """Read process output and update the log display"""
    global sync_process, stop_log_thread
    
    while not stop_log_thread and sync_process is not None:
        try:
            # Read up to 1000 chars at a time from the process output
            output = sync_process.stdout.readline()
            if output:
                # Update the log display
                log_output.value += output
                
                # Keep only the last 20 lines to avoid excessive memory usage
                lines = log_output.value.split('\n')
                if len(lines) > 20:
                    log_output.value = '\n'.join(lines[-20:])
            
            # Check if process has terminated
            if sync_process.poll() is not None:
                log_output.value += "\nSync process terminated.\n"
                sync_process = None
                start_btn.disabled = False
                stop_btn.disabled = True
                status_output.value = "<div style='color: orange; font-weight: bold;'>Sync process terminated</div>"
                break
            
            time.sleep(0.1)
            
        except Exception as e:
            log_output.value += f"\nError monitoring logs: {str(e)}\n"
            break

# Create UI components
start_btn = widgets.Button(description="Start Sync", button_style="success")
stop_btn = widgets.Button(description="Stop Sync", button_style="danger")
stop_btn.disabled = True

status_output = widgets.HTML(value="<div style='color: gray;'>Sync not running</div>")
log_output = widgets.Textarea(value="", layout=widgets.Layout(width='100%', height='200px'))

# Set up event handlers
start_btn.on_click(lambda b: start_sync())
stop_btn.on_click(lambda b: stop_sync())

# Display the UI
display(widgets.HBox([start_btn, stop_btn]))
display(status_output)
display(widgets.Label("Log output:"))
display(log_output)

HTML(value="<div style='color: gray;'>Sync not running</div>")

Label(value='Log output:')

Textarea(value='', layout=Layout(height='200px', width='100%'))